In [9]:
"""#3 k명의 이웃을 뽑아주는 함수 get_k_neighbors 구하기"""
import pandas as pd
import numpy as np
from math import sqrt

RATING_DATA_PATH = 'C:/Users/공성식/Desktop/WORKSTATION/Python Workplace/codeit/intermediate_machine_learning/5fb0e111933f7147701779cf_3681/3681/data/ratings.csv'  # 받아올 평점 데이터 경로 정의

np.set_printoptions(precision=2)  # 소수점 둘째 자리까지만 출력

def distance(user_1, user_2):
    """유클리드 거리를 계산해주는 함수"""
    return sqrt(np.sum((user_1 - user_2)**2))
    
    
def filter_users_without_movie(rating_data, movie_id):
    """movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외해주는 함수"""
    return rating_data[~np.isnan(rating_data[:,movie_id])]
    
    
def fill_nan_with_user_mean(rating_data):
    """평점 데이터의 빈값들을 각 유저 평균 값으로 체워주는 함수"""
    filled_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=0)  # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[inds] = np.take(row_mean, inds[1])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data
    
    
def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들을 찾아주는 함수"""
    distance_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    for j in range(len(rating_data)):
        if j == user_id:
            distance_data[j][-1] = np.inf
        else:
            distance_data[j][-1] = distance(rating_data[user_id],rating_data[j])
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]
    

# 실행 코드     
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values  # 평점 데이터를 불러온다
filtered_data = filter_users_without_movie(rating_data, 3)  # 3 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
filled_data = fill_nan_with_user_mean(filtered_data)  # 빈값들이 채워진 새로운 행렬을 만든다
user_0_neighbors = get_k_neighbors(0, filled_data, 5)  # 유저 0과 비슷한 5개의 유저 데이터를 찾는다
user_0_neighbors

array([[2.  , 3.5 , 1.33, 5.  , 3.25, 4.  , 2.  , 2.  , 2.  , 2.25, 3.  ,
        4.  , 2.  , 5.  , 4.  , 3.5 , 2.33, 3.67, 4.  , 2.  ],
       [2.  , 5.  , 1.  , 4.  , 3.25, 4.  , 2.  , 3.  , 1.  , 2.25, 1.  ,
        2.33, 2.71, 4.  , 2.5 , 3.5 , 2.  , 3.67, 3.14, 4.  ],
       [2.  , 3.5 , 2.  , 5.  , 3.25, 4.  , 2.  , 3.2 , 2.33, 2.25, 2.2 ,
        2.33, 1.  , 3.33, 2.5 , 3.5 , 3.  , 1.  , 5.  , 2.  ],
       [3.  , 3.5 , 1.33, 5.  , 4.  , 4.  , 2.  , 4.  , 5.  , 3.  , 2.2 ,
        1.  , 2.  , 1.  , 1.  , 3.5 , 2.33, 3.67, 4.  , 2.6 ],
       [1.  , 1.  , 1.33, 1.  , 1.  , 4.  , 2.  , 3.2 , 2.33, 2.25, 2.2 ,
        2.  , 2.71, 3.33, 1.  , 3.5 , 2.33, 3.67, 4.  , 2.6 ]])

In [8]:
filled_data

array([[2.  , 3.5 , 1.33, 4.  , 3.25, 5.  , 2.  , 3.2 , 2.  , 2.25, 4.  ,
        2.33, 1.  , 3.33, 2.5 , 3.5 , 2.33, 5.  , 3.14, 2.6 ],
       [2.  , 3.5 , 1.33, 5.  , 3.25, 4.  , 2.  , 2.  , 2.  , 2.25, 3.  ,
        4.  , 2.  , 5.  , 4.  , 3.5 , 2.33, 3.67, 4.  , 2.  ],
       [2.  , 5.  , 1.  , 4.  , 3.25, 4.  , 2.  , 3.  , 1.  , 2.25, 1.  ,
        2.33, 2.71, 4.  , 2.5 , 3.5 , 2.  , 3.67, 3.14, 4.  ],
       [2.  , 3.  , 1.  , 1.  , 3.25, 3.  , 2.  , 4.  , 2.33, 1.  , 1.  ,
        2.33, 4.  , 4.  , 2.  , 3.5 , 2.33, 5.  , 3.  , 2.6 ],
       [1.  , 1.  , 1.33, 1.  , 1.  , 4.  , 2.  , 3.2 , 2.33, 2.25, 2.2 ,
        2.  , 2.71, 3.33, 1.  , 3.5 , 2.33, 3.67, 4.  , 2.6 ],
       [3.  , 3.5 , 1.33, 5.  , 4.  , 4.  , 2.  , 4.  , 5.  , 3.  , 2.2 ,
        1.  , 2.  , 1.  , 1.  , 3.5 , 2.33, 3.67, 4.  , 2.6 ],
       [2.  , 3.5 , 2.  , 5.  , 3.25, 4.  , 2.  , 3.2 , 2.33, 2.25, 2.2 ,
        2.33, 1.  , 3.33, 2.5 , 3.5 , 3.  , 1.  , 5.  , 2.  ],
       [2.  , 5.  , 1.33, 5.  , 3.  , 4. 

In [4]:
ratings_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id')

In [5]:
#DataFrame을 직접 보면서 어떻게 data를 전처리할 것인가에 대한 인사이트를 얻는다.
ratings_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
user_id,,,,,,,,,,,,,,,,,,,,
0,2.0,3.0,4.0,NaN,2.0,3.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,1.0,NaN,NaN,2.0,5.0,2.0,NaN
1,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,2.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,2.0,NaN,1.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,3.0,3.0,NaN,3.0,3.0,4.0,NaN,NaN
3,NaN,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,5.0,3.0,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0
4,NaN,NaN,NaN,5.0,NaN,NaN,2.0,2.0,2.0,NaN,3.0,4.0,2.0,5.0,4.0,NaN,NaN,NaN,4.0,2.0
5,NaN,4.0,3.0,NaN,NaN,5.0,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,1.0,NaN,4.0,3.0,NaN,5.0
6,3.0,NaN,NaN,NaN,4.0,5.0,2.0,2.0,NaN,3.0,NaN,4.0,NaN,NaN,2.0,4.0,NaN,NaN,NaN,1.0
7,NaN,5.0,1.0,4.0,NaN,NaN,NaN,3.0,1.0,NaN,1.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,4.0
8,NaN,3.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN,1.0,1.0,NaN,4.0,4.0,2.0,NaN,NaN,5.0,3.0,NaN


In [10]:
"""#4 추천시스템까지 구현"""
import pandas as pd
import numpy as np
from math import sqrt

RATING_DATA_PATH = 'C:/Users/공성식/Desktop/WORKSTATION/Python Workplace/codeit/intermediate_machine_learning/5fb0e111933f7147701779cf_3681/3681/data/ratings.csv'  # 받아올 평점 데이터 경로 정의

np.set_printoptions(precision=2)  # 소수점 둘째 자리까지만 출력

def distance(user_1, user_2):
    """유클리드 거리를 계산해주는 함수"""
    return sqrt(np.sum((user_1 - user_2)**2))
    
    
def filter_users_without_movie(rating_data, movie_id):
    """movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외해주는 함수"""
    return rating_data[~np.isnan(rating_data[:,movie_id])]
    
    
def fill_nan_with_user_mean(rating_data):
    """평점 데이터의 빈값들을 각 유저 평균 값으로 체워주는 함수"""
    filled_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=0)  # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[inds] = np.take(row_mean, inds[1])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data
    
    
def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들을 찾아주는 함수"""
    distance_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    # 코드를 쓰세요.
    for i in range(len(distance_data)):
        row = distance_data[i]
        
        if i == user_id:  # 같은 유저면 거리를 무한대로 설정
            row[-1] = np.inf
        else:  # 다른 유저면 마지막 열에 거리 데이터를 저장
            row[-1] = distance(distance_data[user_id][:-1], row[:-1])
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]
    
def predict_user_rating(rating_data, k, user_id, movie_id,):
    """예측 행렬에 따라 유저의 영화 평점 예측 값 구하기"""
    # movie_id 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
    filtered_data = filter_users_without_movie(rating_data, movie_id)
    # 빈값들이 채워진 새로운 행렬을 만든다
    filled_data = fill_nan_with_user_mean(filtered_data)
    # 유저 user_id와 비슷한 k개의 유저 데이터를 찾는다
    neighbors = get_k_neighbors(user_id, filled_data, k)
    neighbors_avg = np.mean(neighbors[:,movie_id])
    
    return neighbors_avg
    
# 실행 코드   
# 평점 데이터를 불러온다
rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values
# 5개의 이웃들을 써서 유저 0의 영화 3에 대한 예측 평점 구하기
predict_user_rating(rating_data, 5, 0, 3)  



4.0